# 02 Price Modeling (P-band)Goal: build **SARIMA**, **ETS**, and **ML** baselines for monthly real prices.We keep the last 12 months as the test set.

In [ ]:
import sysfrom pathlib import Pathimport matplotlib.pyplot as pltimport pandas as pdPROJECT_ROOT = Path("..").resolve()sys.path.append(str(PROJECT_ROOT))from src.models.price_model import (    fit_ets,    fit_ml_baselines,    fit_sarima,    summarize_metrics,)

## 1) Load monthly price dataWe read the monthly price table and build a simple date column.

In [ ]:
crop = "wheat"data_dir = PROJECT_ROOT / "data"monthly = pd.read_csv(data_dir / f"P-{crop}-monthly.csv")monthly["date"] = pd.to_datetime(    monthly["year"].astype(str)    + "-"    + monthly["month"].astype(str).str.zfill(2)    + "-01")monthly = monthly.sort_values("date")monthly.head()

## 2) Quick look at the target

In [ ]:
plt.figure(figsize=(10, 4))plt.plot(monthly["date"], monthly["price_real"], label="price_real")plt.title(f"Monthly Real Price - {crop.title()}")plt.xlabel("Date")plt.ylabel("Real price")plt.grid(True, alpha=0.3)plt.legend()plt.tight_layout()plt.show()

## 3) SARIMA and ETSThese are classic time-series models that use only the price history.

In [ ]:
# Last 12 months as testsarima_res = fit_sarima(crop=crop, test_size=12, data_dir=data_dir)ets_res = fit_ets(crop=crop, test_size=12, data_dir=data_dir)metrics_ts = summarize_metrics([sarima_res, ets_res])metrics_ts

In [ ]:
plt.figure(figsize=(10, 4))plt.plot(sarima_res.test_index, sarima_res.y_true, label="actual", color="black")plt.plot(sarima_res.test_index, sarima_res.y_pred, label="sarima")plt.plot(ets_res.test_index, ets_res.y_pred, label="ets")plt.title(f"Test Forecasts - {crop.title()}")plt.xlabel("Date")plt.ylabel("Real price")plt.grid(True, alpha=0.3)plt.legend()plt.tight_layout()plt.show()

## 4) ML baselinesWe use the prepared feature table and predict the next month's real price.

In [ ]:
ml_results = fit_ml_baselines(crop=crop, test_size=12, data_dir=data_dir)metrics_ml = summarize_metrics(ml_results.values())metrics_ml

In [ ]:
best_model_name = metrics_ml.iloc[0]["model"]best_model = ml_results[best_model_name]plt.figure(figsize=(10, 4))plt.plot(best_model.test_index, best_model.y_true, label="actual", color="black")plt.plot(best_model.test_index, best_model.y_pred, label=f"ml: {best_model_name}")plt.title(f"ML Forecast - {crop.title()}")plt.xlabel("Date")plt.ylabel("Real price")plt.grid(True, alpha=0.3)plt.legend()plt.tight_layout()plt.show()

## 5) Quick multi-crop comparison (optional)This cell runs the same models for all crops and compares errors.

In [ ]:
rows = []for crop_name in ["wheat", "barley", "maize"]:    sarima_res = fit_sarima(crop=crop_name, test_size=12, data_dir=data_dir)    ets_res = fit_ets(crop=crop_name, test_size=12, data_dir=data_dir)    ml_results = fit_ml_baselines(crop=crop_name, test_size=12, data_dir=data_dir)    metrics = summarize_metrics([sarima_res, ets_res, *ml_results.values()])    metrics.insert(0, "crop", crop_name)    rows.append(metrics)summary = pd.concat(rows, ignore_index=True)summary